In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score

In [ ]:
df = pd.read_csv('data/dataset_20210515.csv')
df_champions = pd.read_csv('data/dataset-champions.csv')
df

In [ ]:
y = df['result'].copy()
X = df.drop('result',axis=1)
X

In [ ]:
best_features_list = ['bluetop','bluetopgp','bluetopwr','bluetopkda','bluejungle','bluejunglegp','bluejunglewr','bluejunglekda','bluemid','bluemidgp','bluemidwr','bluemidkda','blueadc','blueadcgp','blueadcwr','blueadckda','bluesupport','bluesupportgp','bluesupportwr','bluesupportkda','redtop','redtopgp','redtopwr','redtopkda','redjungle','redjunglegp','redjunglewr','redjunglekda','redmid','redmidgp','redmidwr','redmidkda','redadc','redadcgp','redadcwr','redadckda','redsupport','redsupportgp','redsupportwr','redsupportkda']
new_X = X[best_features_list]

new_X['bluetopwr'] = new_X['bluetopwr'].str.rstrip('%').astype('float') / 100.0 if isinstance(new_X['bluetopwr'], str) else 0
new_X['bluejunglewr'] = new_X['bluejunglewr'].str.rstrip('%').astype('float') / 100.0 if isinstance(new_X['bluejunglewr'], str) else 0
new_X['bluemidwr'] = new_X['bluemidwr'].str.rstrip('%').astype('float') / 100.0 if isinstance(new_X['bluemidwr'], str) else 0
new_X['blueadcwr'] = new_X['blueadcwr'].str.rstrip('%').astype('float') / 100.0 if isinstance(new_X['blueadcwr'], str) else 0
new_X['bluesupportwr'] = new_X['bluesupportwr'].str.rstrip('%').astype('float') / 100.0 if isinstance(new_X['bluesupportwr'], str) else 0

new_X['redtopwr'] = new_X['redtopwr'].str.rstrip('%').astype('float') / 100.0 if isinstance(new_X['redtopwr'], str) else 0
new_X['redjunglewr'] = new_X['redjunglewr'].str.rstrip('%').astype('float') / 100.0 if isinstance(new_X['redjunglewr'], str) else 0
new_X['redmidwr'] = new_X['redmidwr'].str.rstrip('%').astype('float') / 100.0 if isinstance(new_X['redmidwr'], str) else 0
new_X['redadcwr'] = new_X['redadcwr'].str.rstrip('%').astype('float') / 100.0 if isinstance(new_X['redadcwr'], str) else 0
new_X['redsupportwr'] = new_X['redsupportwr'].str.rstrip('%').astype('float') / 100.0 if isinstance(new_X['redsupportwr'], str) else 0

for index, row in new_X.iterrows():
    new_X.loc[index, 'bluetop'] = int(df_champions.loc[df_champions['id'] == row['bluetop']]['key'])    
    new_X.loc[index, 'bluejungle'] = int(df_champions.loc[df_champions['id'] == row['bluejungle']]['key'])
    new_X.loc[index, 'bluemid'] = int(df_champions.loc[df_champions['id'] == row['bluemid']]['key'])    
    new_X.loc[index, 'blueadc'] = int(df_champions.loc[df_champions['id'] == row['blueadc']]['key'])
    new_X.loc[index, 'bluesupport'] = int(df_champions.loc[df_champions['id'] == row['bluesupport']]['key'])

    new_X.loc[index, 'redtop'] = int(df_champions.loc[df_champions['id'] == row['redtop']]['key'])
    new_X.loc[index, 'redjungle'] = int(df_champions.loc[df_champions['id'] == row['redjungle']]['key'])
    new_X.loc[index, 'redmid'] = int(df_champions.loc[df_champions['id'] == row['redmid']]['key'])
    new_X.loc[index, 'redadc'] = int(df_champions.loc[df_champions['id'] == row['redadc']]['key'])
    new_X.loc[index, 'redsupport'] = int(df_champions.loc[df_champions['id'] == row['redsupport']]['key'])


# blue_team = new_X[['bluetop','bluejungle', 'bluemid', 'blueadc', 'bluesupport']].values
# red_team = new_X[['redtop', 'redjungle', 'redmid', 'redadc', 'redsupport']].values
new_X

In [ ]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42)
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(new_X,y)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Support Vector Machine (Linear Kernel)': LinearSVC(),
    'Support Vector Machine (RBF Kernel)': SVC(),
    'Decission Tree': DecisionTreeClassifier(),
    'Adaboost': AdaBoostClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier()
}

for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

In [ ]:
scores_list = []
for name,model in models.items():
    scores_list.append({
    'Model': name,
    'Accuracy': accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test))
    })
scores = pd.DataFrame(scores_list)

In [ ]:
scores